In [1]:
import os
import sys
# //*** Imports and Load Data
#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json

#//*** Use the whole window in the IPYNB editor
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#//*** Maximize columns and rows displayed by pandas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

install plaidml: https://towardsdatascience.com/plaidml-deep-learning-on-a-budget-ce22b58ba8c2

LTSM Code: https://medium.com/analytics-vidhya/beginners-guide-to-stock-prediction-using-lstm-7010bf8b8c21

In [2]:
#//*** Setup plaid keras
import plaidml.keras

import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

plaidml.keras.install_backend()


import keras

In [3]:
import keras
import keras.backend as K

K

<module 'plaidml.keras.backend' from 'C:\\Users\\stonk013\\Anaconda3\\envs\\plaid-ml\\lib\\site-packages\\plaidml\\keras\\backend.py'>

In [4]:
# 


#importing the libraries
import math
#import matplotlib.pyplot as plt
#import seaborn
import pandas as pd
import numpy as np
import datetime
import time
#import requests
import io
from sklearn.preprocessing import MinMaxScaler
#from statsmodels.tsa.arima_model import ARIMA
#import statsmodels.api as sm
#import pandas_datareader as web
from sklearn.metrics import mean_squared_error
#Importing the keras model
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

from keras.layers import LSTM
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor

In [5]:
#data_df = web.DataReader('APOLLOHOSP.NS', data_source = 'yahoo', start="2016-05-01", end="2021-05-01")
data_df = pd.read_csv("amc_daily.csv.zip")
data_df.head(10)

,date,symbol,open,high,low,close,volume
0,2021-06-30,amc,56.00,58.180,54.65,56.68,59020626
1,2021-06-29,amc,59.06,61.000,56.18,56.43,63604135
2,2021-06-28,amc,55.10,59.360,54.33,58.11,99310240
3,2021-06-25,amc,55.75,56.290,52.97,54.06,77596927
4,2021-06-24,amc,57.98,58.760,55.66,56.70,80351219
5,2021-06-23,amc,57.04,61.100,56.80,58.30,116291804
6,2021-06-22,amc,54.10,58.740,51.05,58.27,169494144
7,2021-06-21,amc,61.34,63.010,53.43,55.69,185876142
8,2021-06-18,amc,61.26,64.955,56.85,59.26,243645937
9,2021-06-17,amc,54.00,63.830,52.98,60.73,303576010


In [7]:
#Create a new dataframe with only the 'Adj close column'
new_data = data_df.filter(['close'])
#Convert the dataframe to a numpy array
dataset = new_data.to_numpy()
#Get the number of rows to train the model on
len_train = math.ceil(len(dataset)*.8)

train = dataset[0:len_train, :]
test = dataset[len_train:len(dataset), :1]

In [8]:
##Feature Scaling
sc = MinMaxScaler(feature_range = (0, 1)) 
train_scaled = sc.fit_transform(train.reshape(-1,1))
test_scaled = sc.fit_transform(test.reshape(-1,1))

In [9]:
X_train, y_train = [], []
for i in range(len(train)-80-1):
    a = dataset[i:(i+80), 0]
    X_train.append(a)
    y_train.append(dataset[i+80, 0])
X_train = np.array(X_train)
y_train = np.array(y_train)

In [10]:
#Reshaping
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
#X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
X_test = test.reshape(test.shape[0], test.shape[1], 1)

In [11]:
#create and fit the LSTM network
def model_lstm(lrn_rate = 0.01, mntum = 0):
    #Intializing the RNN
    model = Sequential()
    #1st layer
    model.add(LSTM(50, return_sequences=True, input_shape = (X_train.shape[1], 1)))
    model.add(Dropout(0.2))
    #2nd layer
    model.add(LSTM(50))
    model.add(Dropout(0.2))
    #Adding the output layer
    model.add(Dense(units = 1))
    model.compile(loss = 'mean_squared_error', optimizer='adam')
    return model

In [19]:
start_time = time.time()
LSTModel = model_lstm(lrn_rate=0.01, mntum=0)
LSTMmodel_fit = LSTModel.fit(X_train, y_train, batch_size=1, epochs=10)
print(f"Run time internal GPU: {time.time()-start_time}")

INFO:plaidml:Opening device "opencl_nvidia_quadro_k1200.0"


Epoch 1/10


INFO:plaidml:Analyzing Ops: 3590 of 15581 operations complete
INFO:plaidml:Analyzing Ops: 9293 of 15581 operations complete
INFO:plaidml:Analyzing Ops: 14315 of 15581 operations complete


1436/1436 [==============================] - 222s 154ms/step - loss: 49.6085
Epoch 2/10
1436/1436 [==============================] - 168s 117ms/step - loss: 5.4986
Epoch 3/10
1436/1436 [==============================] - 167s 117ms/step - loss: 4.2215
Epoch 4/10
1436/1436 [==============================] - 176s 123ms/step - loss: 3.4787
Epoch 5/10
1436/1436 [==============================] - 171s 119ms/step - loss: 4.0885
Epoch 6/10
1436/1436 [==============================] - 170s 118ms/step - loss: 3.6799
Epoch 7/10
1436/1436 [==============================] - 170s 118ms/step - loss: 3.5377
Epoch 8/10
1436/1436 [==============================] - 169s 118ms/step - loss: 3.7995
Epoch 9/10
1436/1436 [==============================] - 170s 118ms/step - loss: 3.6500
Epoch 10/10
1436/1436 [==============================] - 170s 118ms/step - loss: 3.5054
Run time internal GPU: 1754.7859437465668


In [ ]:
start_time = time.time()
LSTModel = model_lstm(lrn_rate=0.01, mntum=0)
LSTMmodel_fit = LSTModel.fit(X_train, y_train, batch_size=1, epochs=10)
print(f"Run time CPU PC2: {time.time()-start_time}")

INFO:plaidml:Opening device "llvm_cpu.0"


Epoch 1/10


In [21]:
closing_price = LSTModel.predict(X_test)
closing_price = sc.inverse_transform(closing_price)
closing_price

ValueError: Error when checking input: expected lstm_1_input to have shape (80, 1) but got array with shape (1, 1)

In [19]:
rms=np.sqrt(np.mean(np.power((y1-closing_price),2)))
rms

NameError: name 'y1' is not defined